In [3]:
%%configure -f
{
    // You can get a list of valid parameters to config the session from https://github.com/cloudera/livy#request-body.
    "driverMemory": "2g", // Recommended values: ["28g", "56g", "112g", "224g", "400g", "472g"]
    "driverCores": 2, // Recommended values: [4, 8, 16, 32, 64, 80]
    "executorMemory": "2g",
    "executorCores": 2,
    "numExecutors": 1,
    "conf":
    {
        // Example of standard spark property, to find more available properties please visit: https://spark.apache.org/docs/latest/configuration.html#application-properties.
        "spark.driver.maxResultSize": "10g",
        // Example of customized property, you can specify count of lines that Spark SQL returns by configuring "livy.rsc.sql.num-rows".
        "livy.rsc.sql.num-rows": "3000"
    }
}

In [2]:
val paramUserName = "notebookUser"
val paramBasePath = "/experiment/notebookBasePath"

In [3]:
import com.tccc.dna.synapse._
import com.tccc.dna.synapse.spark._
import com.tccc.dna.supplychain.cs.utils.AdminUtils

## Print version and runtime info

In [4]:
SynapseSpark.printVersions
println()
println("Total provisioned executors: " + SynapseSpark.activeExecutorCount)
println()
println("Total provisioned cores: " + SynapseSpark.totalClusterExecutorCores)

## Create root folders

In [5]:
//basePath will be converted to lowercase as a standard convention
val rootSetupResults = AdminUtils.createRootFolders(basePath = paramBasePath)

In [6]:
  def ifFileExists(filePath: String): Boolean = {
    var exists = false
    try {
      exists = if(mssparkutils.fs.ls(filePath)==null) false else true
    } catch {
      case e: Exception => println(s"Informational message only! [$filePath] does not exist so creating it.")
    }
    exists
  }

ifFileExists("/experiment/notebookbasepath/data")
AzStorage.ifFileExists("/experiment/notebookbasepath/data")

ifFileExists("experiment")
AzStorage.ifFileExists("experiment")

val relativePath = "/experiment/data/certified"
    val fullPath = "abfss://default@xxx.dfs.core.windows.net/experiment/data/certified/"

    if(AzStorage.ifFileExists(relativePath)) println(s"Relative Path [$relativePath] exists.")
    if(AzStorage.ifFileExists(fullPath)) println(s"Full Path [$fullPath] exists.")

    if(ifFileExists(relativePath)) println(s"Relative Path [$relativePath] exists.")
    if(ifFileExists(fullPath)) println(s"Full Path [$fullPath] exists.")

## Create experimentation structure for user: Aravind

In [7]:
  import scala.collection.mutable
  def createFolders(paths: List[String]): mutable.SortedMap[String, String] = {
    val result = mutable.SortedMap[String, String]()
    paths.foreach(path => {
      if (AzStorage.ifFileExists(path)) {
        result += (path -> "Folder already exists. No need to create!")
      } else {
        val isSuccess = mssparkutils.fs.mkdirs(path)
        result += (path -> s"Created successfully? = ${isSuccess.toString}")
      }
    })
    result
  }

 val ExperimentFolder: String = "/experiment"
 val DataZoneFolders: List[String] = List("stage", "refined", "certified")

  def createUserFolders(userName: String): mutable.SortedMap[String, String] = {
    val basePath = ExperimentFolder + "/" + userName.toLowerCase + "/"
    val userExpFolders = DataZoneFolders.map(f => basePath + f)

    val results = createFolders(userExpFolders)

    for ((folder, result) <- results) {
      val msg = s"Folder: $folder, Result: $result"
      //logInfo(msg)
      println(msg) //output to notebook cell
    }

    results
  }

In [8]:
//userName will be converted to lowercase as a standard convention AdminUtils.
val userSetupResults = createUserFolders(userName = paramUserName)

In [1]:
val isSuccess = mssparkutils.fs.mkdirs("/experiment/notebookuser")

## Create data zones

In [9]:
//basePath will be converted to lowercase as a standard convention
val userZoneSetupResults = AdminUtils.createDataZones(basePath = paramBasePath)

In [10]:
println("Total provisioned executors: " + SynapseSpark.activeExecutorCount)
println()
println("Total provisioned cores: " + SynapseSpark.totalClusterExecutorCores)

In [10]:
val notebookFinalOutput = rootSetupResults.mkString("\n").concat("\n\n")
                          .concat(userSetupResults.mkString("\n")).concat("\n\n")
                          .concat(userZoneSetupResults.mkString("\n"))

mssparkutils.notebook.exit(notebookFinalOutput)